In [1]:
!pip install -U bitsandbytes

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
import re
from collections import Counter
import random
from transformers import pipeline, set_seed

In [3]:
from google.colab import files
import os
required_files = ["prefiksy.txt"]
for f in required_files:
    if not os.path.exists(f):
        files.upload()

In [4]:
with open("prefiksy.txt", "r") as f:
    prefixes_str = f.read()

In [5]:
prefixes_str = """
Pan Piotr pali
Wieczorem widzieliśmy
"""

In [6]:
prefixes = prefixes_str.splitlines()

In [7]:
prefixes[:5]

['', 'Pan Piotr pali', 'Wieczorem widzieliśmy']

In [8]:
model_name = "eryk-mazus/polka-1.1b-chat"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
    device_map="auto"
).to(device)

streamer = TextStreamer(tokenizer, skip_prompt=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [9]:
model_name2 = "eryk-mazus/polka-1.1b"

model2 = AutoModelForCausalLM.from_pretrained(
    model_name2,
    torch_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
    device_map="auto"
).to(device)

In [10]:
generator = pipeline('text-generation', model="eryk-mazus/polka-1.1b")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [11]:
prefixes

['', 'Pan Piotr pali', 'Wieczorem widzieliśmy']

In [12]:
# Systemowy prompt
system_prompt = (
    # "Przepisujesz podane zdanie i dodajesz logiczny koniec. "
    # "Dostajesz literę i zdanie. Musisz dokończyć zdanie słowami na zaczynającymi się od podanej litery."
    # "Musisz dokończyć dane słowo i zdanie, którego słowa muszą zaczynać się od tej samej litery."
    "Musisz dokończyć dane słowo i zdanie"
)

# chat = [{"role": "system", "content": system_prompt}]
# chat.append({"role": "user", "content": "Dokończ zdanie: Pan Piotr pali"})
# chat.append({"role": "assistant", "content": "Odpowiedź: Pan Piotr pali papierosy."})
# chat.append({"role": "user", "content": "Dokończ zdanie: Karolina kupiła królowi"})
# chat.append({"role": "assistant", "content": "Odpowiedź: Karolina kupiła królowi Karolowi komputer."})
# chat.append({"role": "user", "content": "Dokończ zdanie: Piękna piosenkarka prowadzi"})
# chat.append({"role": "assistant", "content": "Odpowiedź: Piękna piosenkarka prowadzi piękny pomarańczowy pojazd."})
# chat = [{"role": "system", "content": system_prompt}]
# chat.append({"role": "user", "content": "Pan Piotr pali"})
# chat.append({"role": "assistant", "content": "Pan Piotr pali papierosy."})
# chat.append({"role": "user", "content": "Karolina kupiła królowi"})
# chat.append({"role": "assistant", "content": "Karolina kupiła królowi Karolowi komputer."})
# chat.append({"role": "user", "content": "Piękna piosenkarka prowadzi"})
# chat.append({"role": "assistant", "content": "Piękna piosenkarka prowadzi piękny pomarańczowy pojazd."})
# chat = [{"role": "system", "content": system_prompt}]
# chat.append({"role": "user", "content": "Litera: p, zdanie: Pan Piotr pali"})
# chat.append({"role": "assistant", "content": "papierosy."})
# chat.append({"role": "user", "content": "Litera: k, zdanie: Karolina kupiła królowi"})
# chat.append({"role": "assistant", "content": "Karolowi komputer."})
# chat.append({"role": "user", "content": "Litera: p, zdanie: Piękna piosenkarka prowadzi"})
# chat.append({"role": "assistant", "content": "piękny pomarańczowy pojazd."})
chat = [{"role": "system", "content": system_prompt}]
chat.append({"role": "user", "content": "Pan Piotr pali p"})
chat.append({"role": "assistant", "content": "Pan Piotr pali papierosy."})
chat.append({"role": "user", "content": "Karolina kupiła królowi K"})
chat.append({"role": "assistant", "content": "Karolina kupiła królowi Karolowi komputer."})
chat.append({"role": "user", "content": "Piękna piosenkarka prowadzi p"})
chat.append({"role": "assistant", "content": "Piękna piosenkarka prowadzi piękny pomarańczowy pojazd."})

In [13]:
def longest_common_prefix_and_strip(s1, s2):
    prefix = []
    for a, b in zip(s1, s2):
        if a == b:
            prefix.append(a)
        else:
            break
    prefix = ''.join(prefix)

    s1_stripped = s1[len(prefix):]
    s2_stripped = s2[len(prefix):]
    return prefix, s1_stripped, s2_stripped

In [15]:
model_name_2 = "eryk-mazus/polka-1.1b"

tokenizer = AutoTokenizer.from_pretrained(model_name_2, padding_side="left")
tokenizer.pad_token = tokenizer.eos_token


model_2 = AutoModelForCausalLM.from_pretrained(model_name_2, device_map="auto", load_in_8bit=True).to(device)

prompt_2 = """Przykładowe zapytanie do modelu"""

model_inputs = tokenizer([prompt_2], return_tensors="pt").to("cuda")
with torch.no_grad():
  generated_ids = model_2.generate(
    **model_inputs,
    max_new_tokens=512,
    do_sample=True,
    penalty_alpha=0.6,
    top_k=5
  )

output = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(output)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


KeyboardInterrupt: 

In [ ]:
# prompt = f"Dokończ zdanie: Karolina kocha karola"
p = random.choice(prefixes)
sentences = []
print(p, len(p), "prompt")
letter = p[0].lower()
print("---------")
prompt = f"{p} {letter}"
# prompt = p
chat.append({"role": "user", "content": prompt})
inputs = tokenizer.apply_chat_template(chat, add_generation_prompt=True, return_tensors="pt")
# first_param_device = next(model.parameters()).device
# inputs = inputs.to(first_param_device)
temperature_values = [0.2, 0.3, 0.4]
top_p_values = [0.8, 0.9, 0.95]
for _ in range(3):
    for t in temperature_values:
        for top_p in top_p_values:
            # outputs = model.generate(
            # inputs,
            # pad_token_id=tokenizer.eos_token_id,
            # max_new_tokens=int(len(p) * 1.5),
            # temperature=t,
            # repetition_penalty=1.5,
            # top_p=top_p,
            # do_sample=True,
            # # streamer=streamer,
            # )
            g = generator(prompt, pad_token_id=generator.tokenizer.eos_token_id)[0]['generated_text']
            print(g)
            # new_tokens = outputs[0, inputs.size(1):]
            # response = tokenizer.decode(new_tokens, skip_special_tokens=True)
            # prefix, s1_stripped, s2_stripped = longest_common_prefix_and_strip(p, response)
            # print(response)
            # if prefix == p:
            #     for w in s2_stripped.split():
            #         if w[0].lower() == letter:
            #             sentences.append(f"{p} {w}")
            #         else:
            #             print(response, "odp")
            #             print(s2_stripped, "!!!!")
            #             break

            # print(sentences, "sent")
print(response, "odp")
print("---------")
print(prefix == p)
print("---------")
print(s2_stripped, "!!!!")
print("---------")
